# OpenCV Guide
This is a tool that will utilise the OpenCV image processing tools to collate images and information for data that is present in images.

In [2]:
import os
import sys
import cv2
import numpy as np
import tensorflow as tf

from google.colab.patches import cv2_imshow

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
import keras.utils as image
import matplotlib.pyplot as plt

def read_images(path, sz=None):
    """Reads the images in a given folder, resizes images on the fly if size is given.

    Args:
        path: Path to a folder with subfolders representing the subjects (persons).
        sz: A tuple with the size Resizes

    Returns:
        A list [X,y]

            X: The images, which is a Python list of numpy arrays.
            y: The corresponding labels (the unique number of the subject, person) in a Python list.
    """
    c = 0
    strerror = ''
    errno = 0
    X,y = [], []
    new_size = (sz, sz)
    for dirname, dirnames, filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname, subdirname)
            for filename in os.listdir(subject_path):
                try:
                    if (filename == ".directory"):
                        continue
                    filepath = os.path.join(subject_path, filename)
                    print (filepath)
                    im = image.load_img(os.path.join(subject_path, filename))
                    
                    print(im)
                    plt.imshow(im)
                    if (im is None):
                        print ("image " + filepath + " is none") 
                    # resize to given size (if given)
                    if (sz is not None):   
                        #ref: https://www.geeksforgeeks.org/image-recognition-with-mobilenet/
                        res_img = im.resize(new_size)
                        # Convert the PIL image to Tensor
                        resizedimg = image.img_to_array(res_img)
                        finalimg = np.expand_dims(resizedimg,axis=0)
                        finalimg = tf.keras.applications.mobilenet_v2.preprocess_input(finalimg)
                        finalimg.shape                       

                    #X.append(np.asarray(im, dtype=np.uint8))
                    X.append(finalimg)
                    y.append(c)
                # except IOError, (errno, strerror):
                #     print ("I/O error({0}): {1}".format(errno, strerror))
                except:
                    print ("Unexpected error:", sys.exc_info()[0])
                    raise
            c = c+1
    return [X,y]


The read_images function expects that the folder structure in the Google Drive has images folder followed by two child folders for two classes (as in the model classification). Therefore, it could be a cat folder and a dog folder for cat| dog classification exercise. 

Note, it is able to read the child folders automatically. And derive the images from there and place them in X. So, X would contain, 10 cats and 10 dogs images. While Y would contain 20 answers for these 20 images.

In [ ]:
[X,y] = read_images('/content/drive/MyDrive/OpenCV/images/', 160)

In [ ]:
y

`#Always good to test it on one image to check the connectivity between Google Drive and Colab. If the test succeeds then repeat the exercise for video processing.`

In [ ]:
#cv2_imshow(X)

Load the h5 pre-trained model file from Google Drive. Ensure it is present at that folder or update the path in the statement below

In [ ]:
from tensorflow.keras.models import load_model
import tensorflow as tf
model = tf.keras.models.load_model('/content/drive/MyDrive/OpenCV/trained_model/my_model.h5')

In [ ]:
print("Total Params:", model.count_params())
model.summary()

model.predict can be used in a loop to predict the answer for all images from test dataset. The classification after compared can go in a CSV file to record the answer for the given image.

When proprocessing is needed for subsequent models then the script remains the same except for the CSV file name, and the preprocessing of image to be predicted by the model. 

In [ ]:
#predicted_labels = model.predict(X[0])
model.predict(X[0])
#print("no helmet" if model.predict(X[0]) > 0 else "helmet")

1/1 [==============================] - 1s 1s/step


array([[-0.24773629]], dtype=float32)